# Defiltering for deblurring. 
Experiments with five defiltering schemes proposed initially for the problem of deblurring noisy images. 

## Usual Python imports 

In [ ]:
import numpy as np
from PIL import Image
import skimage
import skimage.color
import scipy.signal
import scipy.fft
import matplotlib.pyplot as plt

## Some helper functions (psf2otf, zero padding, estimate noise, ...)

In [ ]:
def zero_pad(image, shape, position='corner'):
    shape = np.asarray(shape, dtype=int)
    imshape = np.asarray(image.shape, dtype=int)

    if np.alltrue(imshape == shape):
        return image

    if np.any(shape <= 0):
        raise ValueError("ZERO_PAD: null or negative shape given")

    dshape = shape - imshape
    if np.any(dshape < 0):
        raise ValueError("ZERO_PAD: target size smaller than source one")

    pad_img = np.zeros(shape, dtype=image.dtype)

    idx, idy = np.indices(imshape)

    if position == 'center':
        if np.any(dshape % 2 != 0):
            raise ValueError("ZERO_PAD: source and target shapes "
                             "have different parity.")
        offx, offy = dshape // 2
    else:
        offx, offy = (0, 0)

    pad_img[idx + offx, idy + offy] = image

    return pad_img

In [ ]:
def psf2otf(psf, shape):
    if np.all(psf == 0):
        return np.zeros_like(psf)

    pad = False

    if len(shape) == 3:
        pad = True
        orig_shape = shape
        shape = (shape[0], shape[1])

    inshape = psf.shape
    psf = zero_pad(psf, shape, position='corner')

    for axis, axis_size in enumerate(inshape):
        psf = np.roll(psf, -int(axis_size / 2), axis=axis)

    if pad:
        psf2 = np.zeros(orig_shape)
        psf2[:,:,0] = psf
    else:
        psf2 = psf

    #otf = np.fft.fft2(psf)
    #otf = scipy.fft.fft2(psf)
    otf = scipy.fft.fftn(psf2)

    n_ops = np.sum(psf.size * np.log2(psf.shape))
    otf = np.real_if_close(otf, tol=n_ops)

    return otf

In [ ]:
def estimate_noise(I):
    H = I.shape[0]
    W = I.shape[1]
    M = np.array([[1, -2, 1], [-2, 4, -2], [1, -2, 1]])
    S = np.sum(np.sum(np.abs(scipy.signal.convolve2d(I, M))))
    S = S*np.sqrt(0.5*np.pi)/(6.0*(W-2.0)*(H-2.0))
    return S 

In [ ]:
def estimate_nsr(I):
    I = skimage.color.rgb2gray(skimage.color.rgba2rgb(I)) if len(I.shape) == 3 else I
    en = estimate_noise(I)
    nsr = en**2 / np.var(I[:])
    return nsr

In [ ]:
def reflect(I):
    if len(I.shape) == 3:
        Is = I[::-1, ::-1, :]
    else:
        Is = I[::-1, ::-1]
    return Is

## Phase corrected Van Cittert

In [ ]:
def pcVC(F, y, maxiter=100):
    TM = y.copy()
    
    for i in range(maxiter):
        H = scipy.fft.fftn(F(TM)) / (scipy.fft.fftn(TM)+2.2204e-16)
        TM = TM + np.real(scipy.fft.ifftn((scipy.fft.fftn(y)/(H+2.2204e-16)-scipy.fft.fftn(TM)) * np.absolute(H)))
    return TM 

In [ ]:
def pcVC_nsr(F, y, maxiter=100):
    # Use the same formulation as in the paper 
    nsr = estimate_nsr(y)
    a = 100.0*nsr 
    TM = y.copy()
    
    for i in range(maxiter):
        H = scipy.fft.fftn(F(TM)) / (scipy.fft.fftn(TM)+2.2204e-16)
        Hconj = np.conjugate(H)
        TM = TM + np.real(scipy.fft.ifftn(Hconj/(np.absolute(H)+a) * (scipy.fft.fftn(y) - H*scipy.fft.fftn(TM))))
    return TM 

## Modified LM approach

In [ ]:
def mLM(F, y, maxiter=100):
    nsr = estimate_nsr(y)
    a = 100.0*nsr 
    lm = y.copy()
     
    for i in range(maxiter):
        Flm = F(lm)
        H = scipy.fft.fftn(Flm) / (scipy.fft.fftn(lm) + 1e-16)
        Hconj = np.conjugate(H)
        num = Hconj * (scipy.fft.fftn(y - Flm))
        denom = (Hconj * H + a)
        lm = lm + np.real(scipy.fft.ifftn(num/denom))

    return lm

## Modified Wiener

In [ ]:
def mW(F, y, maxiter=100):
    nsr = estimate_nsr(y)
    #a = 100.0*nsr
    a = nsr
    W = y.copy()
    FW = F(W)
    H = scipy.fft.fftn(FW) / (scipy.fft.fftn(W) + 1e-16)
    for i in range(1, maxiter+1):
        H = H*(i-1)/i + scipy.fft.fftn(FW) / (scipy.fft.fftn(W) + 1e-16)/i
        Hconj = np.conjugate(H)
        W = np.real(scipy.fft.ifftn(Hconj/(Hconj*H + a)*scipy.fft.fftn(y)))
        FW = F(W)

    return W

## Approximate Landweber

In [ ]:
def aL(F, y, maxiter=500):
    L = y.copy()
    
    for i in range(maxiter):
        hp = y - F(L)
        hp = reflect(hp)
        d = (F(L+hp) - F(L-hp))/2.0
        d = reflect(d)
        L = L + d
    
    return L

## Modified Richardson-Lucy

In [ ]:
def mRL(F, y, maxiter=500):
    RL = y.copy()

    for i in range(maxiter):
        r1 = F(RL)/(np.abs(y)+1e-16)
        r1 = reflect(r1)
        r2 = F(r1)
        r2 = reflect(r2)
        RL = RL / (np.abs(r2)+1e-16)

        r1 = y / (np.abs(F(RL))+1e-16)
        r1 = reflect(r1)
        r2 = F(r1)
        r2 = reflect(r2)
        RL = RL * r2

    return RL

## Now some experiments with a grayscale image and noisy motion blur

In [ ]:
img = Image.open('barbara_face.png')
xs = np.asarray(img)
xs = skimage.img_as_float(xs)

Definition of the filter (noisy motion blur). Gaussian noise is used, and kernel2 is used for the motion blur. 

In [ ]:
noise_mean = 0
noise_var = 0.00001
img = Image.open('testkernel2.bmp')
h = np.asarray(img)
h = skimage.color.rgb2gray(h) if len(h.shape) == 3 else h
h = skimage.img_as_float(h)
h = h / np.sum(h[:])
N = xs.shape[0]
M = xs.shape[1]
C = 1 if len(xs.shape)!=3 else xs.shape[2]

Hf = psf2otf(h, (N,M)) if C == 1 else psf2otf(h, (N,M,C))

if C == 1:
    f = lambda x: np.real(scipy.fft.ifft2(scipy.fft.fft2(x[:,:])*Hf))
else:
    f = lambda x: np.real(scipy.fft.ifftn(scipy.fft.fftn(x[:,:,:])*Hf))

F = lambda x: skimage.util.random_noise(f(x), mode='gaussian', mean = noise_mean, var = noise_var)

y = F(xs)

The input and the blurred input image

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(18,6))
axes[0].imshow(xs, cmap='gray', vmin=0.0, vmax=1.0)
axes[0].set_title('Original image')
axes[1].imshow(y, cmap='gray', vmin=0.0, vmax=1.0)
axes[1].set_title('Observed image')
plt.show()

Input, phase corrected VC and LM 

In [ ]:
pcvc = pcVC(F, y)
mlm = mLM(F, y)
mw = mW(F, y)
al = aL(F, y)
mrl = mRL(F, y)

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(18,6))
axes[0].imshow(pcvc, cmap='gray', vmin=0.0, vmax=1.0)
axes[0].set_title('pcVC')
axes[1].imshow(mlm, cmap='gray', vmin=0.0, vmax=1.0)
axes[1].set_title('mLM')
axes[2].imshow(mw, cmap='gray', vmin=0.0, vmax=1.0)
axes[2].set_title('mW')
plt.show()

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(18,6))
axes[0].imshow(xs, cmap='gray', vmin=0.0, vmax=1.0)
axes[0].set_title('Original image')
axes[1].imshow(al, cmap='gray', vmin=0.0, vmax=1.0)
axes[1].set_title('aL')
axes[2].imshow(mrl, cmap='gray', vmin=0.0, vmax=1.0)
axes[2].set_title('mRL')
plt.show()

## Dealing with color images

In [ ]:
img = Image.open('parrots.png')
xs = np.asarray(img)
xs = skimage.img_as_float(xs)

In [ ]:
noise_mean = 0
noise_var = 0.00001
img = Image.open('testkernel2.bmp')
h = np.asarray(img)
h = skimage.color.rgb2gray(h) if len(h.shape) == 3 else h
h = skimage.img_as_float(h)
h = h / np.sum(h[:])
N = xs.shape[0]
M = xs.shape[1]
C = 1 if len(xs.shape)!=3 else xs.shape[2]

Hf = psf2otf(h, (N,M)) if C == 1 else psf2otf(h, (N,M,C))

if C == 1:
    f = lambda x: np.real(scipy.fft.ifft2(scipy.fft.fft2(x[:,:])*Hf))
else:
    f = lambda x: np.real(scipy.fft.ifftn(scipy.fft.fftn(x[:,:,:])*Hf))

F = lambda x: skimage.util.random_noise(f(x), mode='gaussian', mean = noise_mean, var = noise_var)

y = F(xs)

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(18,6))
axes[0].imshow(xs)
axes[0].set_title('Original image')
axes[1].imshow(y)
axes[1].set_title('Observed image')
plt.show()

In [ ]:
pcvc = pcVC(F, y)
mlm = mLM(F, y)
mw = mW(F, y)
al = aL(F, y)
mrl = mRL(F, y)

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(18,6))
axes[0].imshow(pcvc)
axes[0].set_title('pcVC')
axes[1].imshow(mlm)
axes[1].set_title('mLM')
axes[2].imshow(mw)
axes[2].set_title('mW')
plt.show()

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(18,6))
axes[0].imshow(xs)
axes[0].set_title('Original image')
axes[1].imshow(al)
axes[1].set_title('aL')
axes[2].imshow(mrl)
axes[2].set_title('mRL')
plt.show()

## Dealing with color images channel by channel
I.e. for a given color image, apply the defiltering scheme once for each channel. 

In [ ]:
img = Image.open('parrots.png')
xs = np.asarray(img)
xs = skimage.img_as_float(xs)

In [ ]:
noise_mean = 0
noise_var = 0.00001
img = Image.open('testkernel2.bmp')
h = np.asarray(img)
h = skimage.color.rgb2gray(h) if len(h.shape) == 3 else h
h = skimage.img_as_float(h)
h = h / np.sum(h[:])
N = xs.shape[0]
M = xs.shape[1]
C = 1 if len(xs.shape)!=3 else xs.shape[2]

Hf = psf2otf(h, (N,M))

f = lambda x: np.real(scipy.fft.ifft2(scipy.fft.fft2(x[:,:])*Hf))
F = lambda x: skimage.util.random_noise(f(x), mode='gaussian', mean = noise_mean, var = noise_var)

yr = F(xs[:,:,0])
yg = F(xs[:,:,1])
yb = F(xs[:,:,2])
y = np.dstack((yr, yg, yb))

In [ ]:
pcvcr = pcVC(F, yr)
pcvcg = pcVC(F, yg)
pcvcb = pcVC(F, yb)
pcvc = np.dstack((pcvcr, pcvcg, pcvcb))

mlmr = mLM(F, yr)
mlmg = mLM(F, yg)
mlmb = mLM(F, yb)
mlm = np.dstack((mlmr, mlmg, mlmb))

mwr = mW(F, yr)
mwg = mW(F, yg)
mwb = mW(F, yb)
mw = np.dstack((mwr, mwg, mwb))

alr = aL(F, yr)
alg = aL(F, yg)
alb = aL(F, yb)
al = np.dstack((alr, alg, alb))

mrlr = mRL(F, yr)
mrlg = mRL(F, yg)
mrlb = mRL(F, yb)
mrl = np.dstack((mrlr, mrlg, mrlb))

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(18,6))
axes[0].imshow(pcvc)
axes[0].set_title('pcVC')
axes[1].imshow(mlm)
axes[1].set_title('mLM')
axes[2].imshow(mw)
axes[2].set_title('mW')
plt.show()

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(18,6))
axes[0].imshow(xs)
axes[0].set_title('Original image')
axes[1].imshow(al)
axes[1].set_title('aL')
axes[2].imshow(mrl)
axes[2].set_title('mRL')
plt.show()

## Compare pcVC and pcVC_nsr

In [ ]:
img = Image.open('barbara_face.png')
xs = np.asarray(img)
xs = skimage.img_as_float(xs)

In [ ]:
noise_mean = 0
noise_var = 0.00001
img = Image.open('testkernel2.bmp')
h = np.asarray(img)
h = skimage.color.rgb2gray(h) if len(h.shape) == 3 else h
h = skimage.img_as_float(h)
h = h / np.sum(h[:])
N = xs.shape[0]
M = xs.shape[1]
C = 1 if len(xs.shape)!=3 else xs.shape[2]

Hf = psf2otf(h, (N,M)) if C == 1 else psf2otf(h, (N,M,C))

if C == 1:
    f = lambda x: np.real(scipy.fft.ifft2(scipy.fft.fft2(x[:,:])*Hf))
else:
    f = lambda x: np.real(scipy.fft.ifftn(scipy.fft.fftn(x[:,:,:])*Hf))

F = lambda x: skimage.util.random_noise(f(x), mode='gaussian', mean = noise_mean, var = noise_var)

y = F(xs)

In [ ]:
pcvc = pcVC(F, y)
pcvc_nsr = pcVC_nsr(F, y)

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(18,6))
axes[0].imshow(xs, cmap='gray', vmin=0.0, vmax=1.0)
axes[0].set_title('Original image')
axes[1].imshow(pcvc, cmap='gray', vmin=0.0, vmax=1.0)
axes[1].set_title('pcVC')
axes[2].imshow(pcvc_nsr, cmap='gray', vmin=0.0, vmax=1.0)
axes[2].set_title('pcVC with nsr regularization')
plt.show()